<a href="https://colab.research.google.com/github/Paiva-Coppe/Base/blob/main/Acesso_CVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CVM - Acesso dos dados da Comissão de Valores Mobiliários**

**Importação de Pacotes**

In [10]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
#pd.options.display.float_format = "{:.6f}".format
from pandas.tseries.offsets import BDay
from datetime import date
from google.colab import files

###**Funções para Coleta e Tratamento de dados de Fundos de Investimento fornecidos pela CVM**

>*   Base de Dados: http://dados.cvm.gov.br/

> Funções que buscam dados no site da CVM e retornam um DataFrame pandas site:





In [4]:
def busca_cadastro_cvm(data=(date.today()-BDay(1))):
  if data is not busca_cadastro_cvm.__defaults__[0]:
    data = pd.to_datetime(data)
  
  try:
    url = 'http://dados.cvm.gov.br/dados/FI/CAD/DADOS/inf_cadastral_fi_{}{:02d}{:02d}.csv'.format(data.year, data.month, data.day)
    return pd.read_csv(url, sep=';', encoding='ISO-8859-1')

  except: 
    print("Arquivo {} não encontrado!".format(url))
    print("Forneça outra data!")

In [ ]:
#def busca_cadastro_cvm(data=(date.today()-BDay(2))):
  if data is not busca_cadastro_cvm.__default__[0]:
    data = pd.to_datetime(data)

  try:
    url = "http://dados.cvm.gov.br/dados/FI/CAD/DADOS/inf_cadastral_fi_{}{:02d}{:02d}.csv".format(data.year, data.month, data.day)
    return pd.read_csv(url, sep=";", encoding="ISO-8859-1")
    
  except:
    print("Arquivo {} não encontrado!".format(url))
    print("Forneça outra data!")
  

In [ ]:
def busca_informes_diarios_cvm_por_periodo(data_inicio, data_fim):
  datas = pd.date_range(data_inicio, data_fim, freq= "MS")
  informe_completo = pd.DataFrame()

  for data in datas:
    try:
      url = "http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_{}{:02d}.csv".format(data.year, data.month)
      informe_mensal = pd.read_csv(url, sep=";")

    except:
      print("Arquivo {} não encontrado!".format(url))

    informe_completo = pd.concat([informe_completo, informe_mensal], ignore_index=True)

  return informe_completo

In [ ]:
def melhores_e_piores(informes, cadastro, top=5, minimo_de_cotistas=100, classe=""):
  cadastro      = cadastro[cadastro["SIT"] == 'EM FUNCIONAMENTO NORMAL']
  fundos        = informes[informes["NR-COTST"] >= minimo_de_cotistas]
  cnpj_informes = fundos["CNPJ_FUNDO"].DROP_DUPLICATES()

###Buscando Dados da CVM




In [5]:
cadastro = busca_cadastro_cvm()

In [ ]:
cadastro.head(2)

In [9]:
cadastro_cvm = busca_cadastro_cvm()

In [11]:
cadastro_cvm.to_excel("cadastro_cvm_2020-10-27.xlsx")
files.download("cadastro_cvm_2020-10-27.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
informes = busca_informes_diarios_cvm_por_periodo(data_inicio='2020-01', data_fim='2020-10')

In [ ]:
informes


In [ ]:
informes_diarios = busca_informes_cvm(2020,4)
informes_diarios

,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
0,00.017.024/0001-53,2020-04-01,1127984.120000,27.380981,1121661.690000,0.000000,0.000000,1
1,00.017.024/0001-53,2020-04-02,1128125.490000,27.383353,1121758.880000,0.000000,0.000000,1
2,00.017.024/0001-53,2020-04-03,1128288.770000,27.385179,1121833.680000,0.000000,0.000000,1
3,00.017.024/0001-53,2020-04-06,1128434.930000,27.387583,1121932.150000,0.000000,0.000000,1
4,00.017.024/0001-53,2020-04-07,1128235.550000,27.389569,1122013.500000,0.000000,0.000000,1
...,...,...,...,...,...,...,...,...
347356,97.929.213/0001-34,2020-04-24,62585452.400000,8.234617,62577500.240000,0.000000,0.000000,2
347357,97.929.213/0001-34,2020-04-27,62980521.790000,8.286556,62972198.430000,0.000000,0.000000,2
347358,97.929.213/0001-34,2020-04-28,63000881.620000,8.289186,62992186.980000,0.000000,0.000000,2
347359,97.929.213/0001-34,2020-04-29,63218581.900000,8.317784,63209515.120000,0.000000,0.000000,2


In [ ]:
cadastro_cvm

,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,DT_INI_EXERC,DT_FIM_EXERC,CLASSE,DT_INI_CLASSE,RENTAB_FUNDO,CONDOM,FUNDO_COTAS,FUNDO_EXCLUSIVO,TRIB_LPRAZO,INVEST_QUALIF,TAXA_PERFM,INF_TAXA_PERFM,TAXA_ADM,INF_TAXA_ADM,VL_PATRIM_LIQ,DT_PATRIM_LIQ,DIRETOR,CNPJ_ADMIN,ADMIN,PF_PJ_GESTOR,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR
0,36.498.416/0001-29,03 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO,2020-04-09,2020-04-09,NaN,EM FUNCIONAMENTO NORMAL,2020-04-27,2020-04-27,2020-04-27,2020-06-30,Fundo Multimercado,2020-04-09,NaN,Aberto,S,N,S,N,nan,NaN,nan,NaN,150021218.340000,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
1,36.729.776/0001-94,03 MASTER FUNDO DE INVESTIMENTO EM ACOES,2020-04-08,2020-04-08,NaN,FASE PRÉ-OPERACIONAL,2020-04-08,NaN,NaN,NaN,Fundo de Ações,2020-04-08,NaN,Aberto,N,N,NaN,N,nan,NaN,nan,NaN,nan,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
2,36.499.643/0001-79,03 MASTER FUNDO DE INVESTIMENTO MULTIMERCADO,2020-04-09,2020-04-09,NaN,EM FUNCIONAMENTO NORMAL,2020-04-27,2020-04-27,2020-04-27,2020-06-30,Fundo Multimercado,2020-04-09,NaN,Aberto,N,N,S,N,nan,NaN,nan,NaN,149838895.800000,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,09.428.261/0001-81,O3 GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
3,36.016.411/0001-12,051 ALLOCATION FUNDO EM COTAS DE FUNDOS DE INVESTIMENTO MULTIMERCADO CREDITO PRIVADO,2020-04-03,2020-04-02,NaN,FASE PRÉ-OPERACIONAL,2020-04-02,NaN,NaN,NaN,Fundo Multimercado,2020-04-02,NaN,Aberto,S,N,S,S,nan,NaN,nan,NaN,nan,NaN,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM
4,33.913.620/0001-70,051 BRUCUTU FUNDO DE INVESTIMENTO MULTIMERCADO CRÉDITO PRIVADO INVESTIMENTO EXTERIOR,2019-07-19,2019-06-04,NaN,EM FUNCIONAMENTO NORMAL,2019-08-05,2019-08-05,2019-08-05,2020-05-31,Fundo Multimercado,2019-06-04,DI de um dia,Fechado,N,S,NaN,S,20.000000,NaN,0.200000,NaN,24426501.930000,2020-04-29,MARCELO VIEIRA FRANCISCO,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,PJ,30.193.705/0001-33,ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A,62.318.407/0001-19,SANTANDER CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36770,31.340.900/0001-01,ZURICH VALORA PREVIDENCIARIO FI EM COTAS DE FUNDOS DE INVESTIMENTO RENDA FIXA CREDITO PRIVADO,2018-12-19,2018-12-19,NaN,EM FUNCIONAMENTO NORMAL,2019-03-27,2019-03-27,2020-01-01,2020-12-31,Fundo de Renda Fixa,2018-12-19,NaN,Aberto,S,S,N,S,20.000000,O fundo cobrara 20.00 % do que exceder 100.00 % o indice CDIE. A taxa de performance sera paga no(s) mes(es) de Janeiro e Julho,1.100000,NaN,20844690.890000,2020-04-29,ALLAN HADID,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,PJ,07.559.989/0001-17,VALOR

###**Manipulando os dados da CVM**

>Definindo filtros para os Fundos de Investimentos

In [ ]:
minimo_cotistas = 100

>Manipulando os dados e aplicando filtros

In [ ]:
fundos = informes_diarios[informes_diarios['NR_COTST'] >= minimo_cotistas].pivot(index="DT_COMPTC", columns = "CNPJ_FUNDO", values = ["VL_TOTAL",	"VL_QUOTA", "VL_PATRIM_LIQ",	"CAPTC_DIA",  "RESG_DIA"])

In [ ]:
fundos

VL_TOTAL  ...           RESG_DIA
CNPJ_FUNDO 00.068.305/0001-35  ... 97.519.794/0001-36
DT_COMPTC                      ...                   
2020-04-01    52347266.290000  ...      971670.380000
2020-04-02    52286813.890000  ...      434254.390000
2020-04-03    52281604.340000  ...     1015272.400000
2020-04-06    52211795.070000  ...      735636.460000
2020-04-07    52099859.600000  ...      182700.320000
2020-04-08    52013743.910000  ...     2535908.360000
2020-04-09    52007899.940000  ...     1215339.580000
2020-04-13    51833455.310000  ...     2234235.170000
2020-04-14    51858224.530000  ...     1117338.090000
2020-04-15    51857507.070000  ...     1004980.440000
2020-04-16    51832604.490000  ...      368631.570000
2020-04-17    51810701.610000  ...      582196.310000
2020-04-20    51820626.960000  ...      824703.470000
2020-04-22    51794821.720000  ...      327857.260000
2020-04-23    51776604.590000  ...      641327.730000
2020-04-24    51908821.850000  ...     1642396.260000
2020-04-27    51840479.650000  ...      758294.360000
2020-04-28    51899720.190000  ...      612883.790000
2020-04-29    51637623.480000  ...     1217948.080000
2020-04-30    51697882.220000  ...      914635.210000

[20 rows x 13440 columns]

In [ ]:
cotas_normalizadas = fundos["VL_QUOTA"] / fundos["VL_QUOTA"].iloc[0]

In [ ]:
cotas_normalizadas

CNPJ_FUNDO,00.068.305/0001-35,00.071.477/0001-68,00.073.041/0001-08,00.089.915/0001-15,00.180.995/0001-10,00.185.259/0001-54,00.222.725/0001-24,00.222.816/0001-60,00.280.302/0001-60,00.306.278/0001-91,00.322.699/0001-06,00.346.750/0001-10,00.360.293/0001-18,00.398.561/0001-90,00.400.490/0001-13,00.524.617/0001-06,00.539.553/0001-17,00.575.922/0001-27,00.598.452/0001-17,00.601.692/0001-23,00.743.026/0001-20,00.756.851/0001-69,00.777.815/0001-81,00.785.365/0001-79,00.793.947/0001-05,00.807.777/0001-62,00.812.433/0001-41,00.813.340/0001-31,00.813.342/0001-20,00.817.201/0001-86,00.819.855/0001-49,00.819.858/0001-82,00.819.880/0001-22,00.819.889/0001-33,00.819.901/0001-00,00.819.915/0001-23,00.819.929/0001-47,00.822.059/0001-65,00.822.954/0001-80,00.824.198/0001-28,...,47.176.334/0001-84,47.177.431/0001-91,47.177.894/0001-53,47.178.058/0001-93,47.178.215/0001-60,47.178.405/0001-88,47.179.288/0001-77,53.633.558/0001-70,54.069.422/0001-42,54.486.055/0001-82,55.075.238/0001-78,55.234.165/0001-10,56.573.470/0001-07,56.824.857/0001-80,60.906.179/0001-72,63.375.216/0001-51,67.369.363/0001-70,67.845.909/0001-11,67.845.925/0001-04,67.976.423/0001-12,67.976.449/0001-60,68.599.141/0001-06,68.623.479/0001-56,68.670.512/0001-07,68.971.183/0001-26,70.951.678/0001-35,73.232.530/0001-39,73.899.759/0001-21,73.970.642/0001-97,74.055.922/0001-32,74.326.471/0001-20,77.054.658/0001-00,88.002.696/0001-36,88.198.056/0001-43,96.498.654/0001-66,96.498.985/0001-04,96.500.715/0001-82,97.261.093/0001-40,97.519.703/0001-62,97.519.794/0001-36
DT_COMPTC,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,nan,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2020-04-02,0.998858,1.000020,1.000120,0.999818,1.000443,1.019997,1.000136,1.000560,1.000332,0.999894,0.999667,0.996150,0.996866,1.007115,1.004617,1.002695,0.996514,1.019910,1.000118,1.027531,1.000028,1.000074,1.000093,1.000037,0.999699,0.999923,1.000043,0.999654,0.999899,1.000027,1.002509,1.000548,1.002620,1.002623,0.999972,1.000054,0.999977,1.016564,1.002733,1.000098,...,1.017730,1.018934,1.017726,1.017727,1.007144,1.017717,1.010623,1.014691,1.016637,1.013808,1.006323,1.013912,1.013812,1.019354,0.999902,1.014836,0.999683,1.015029,1.017006,1.014732,1.018036,1.000013,0.998858,1.014219,0.999740,1.012609,1.019502,1.018054,0.999861,1.014700,nan,1.016217,1.009473,1.002306,1.016652,1.018095,1.000101,1.013310,0.999693,0.999741
2020-04-03,0.998892,1.000039,1.000242,0.999608,1.000043,0.987041,0.999613,1.000157,1.000165,0.999845,0.999557,0.995508,0.999793,0.976723,1.000755,1.010809,0.995597,0.979841,1.000249,0.973041,1.000072,1.000121,1.000104,1.000082,0.999971,0.999818,1.000085,0.999472,0.999346,1.000062,1.010203,0.998005,1.010438,1.010446,1.000006,1.000169,1.000017,0.981300,1.010927,1.000187,...,0.981082,0.978536,0.981082,0.981108,0.981228,0.981071,0.974379,0.986422,0.980255,0.986106,0.982564,0.986313,0.986090,0.979705,0.999859,0.989576,0.999530,0.989144,0.980435,0.986651,0.979686,1.000033,0.998894,0.936512,0.999818,0.980902,0.985201,0.980109,0.998962,0.986544,nan,0.981735,0.973298,0.969498,0.979965,0.979759,1.000185,0.981742,0.998770,0.999819
2020-04-06,0.998952,1.000062,1.000361,0.999351,0.999896,1.056830,0.999474,1.000132,1.000162,0.999663,0.999191,0.994151,0.996600,1.025714,1.007456,1.005668,0.995024,1.055715,1.000396,1.